***

## Exponential Algorithms - SAT

>CSC427, semester 202 (jan-may 2020)
<br>
10 April 2020
<br><br>
burton rosenberg
<br>
univ of miami
<br>
(c) 2020 all rights reserved

***

We next look at exponential algorithms.

An algorithm solves a problem. It might be an efficient algorithm, full of clever ideas for the 
solution, or an inefficient algorithm but that gets the job done. It does not produce wrong answers,
just might consume more resource than is theoretically needed.

The problem SAT is a problem in solving a boolean formula. By solution, it is meant, that the 
formula has lots of variables and one searches for a setting of the variables True or False so that
the computed value of the formula is True. The values assigned is a called a _satisfying assignment_.

It is a very general problem, because a lot of stuff boils down to a boolean formula. 

However, SAT simplifies the ask in a way which is not essential by restricting the form of the formula.
It must be in _Conjunctive Normal Form_ (CNF). A boolean formula has three operations, AND, OR and NOT. The
AND is also called a conjunction and the OR is also called a disjunction. A disjunct is 
the OR of variables or NOT of a variable,

\begin{eqnarray*}
A &=& a \lor b \lor \neg c \lor e \\
B &=& c \lor \neg b \lor a \\
C &=& b \\
\end{eqnarray*}

Are disjunctions. To satisfy a disjunction, at least one variable be True, if the variable appears simply, 
or False, if the variable appears under a NOT.

Each of these is called a clause. CNF is an AND of a bunch of clauses,

$$
A \land B \land C  = (a \lor b \lor \neg c \lor e)\land (c \lor \neg b \lor a) \land (b) 
$$
A famous NP-complete problem is 3-SAT. It is a problem in finding an assignment of truth values to 
variables so that a certain logical formula evaluates true. The logical formula has a restricted 
format. It must be in _Conjunctive Normal Form_, and furthermore, each conjunct must have exactly 
three terms.

If each clause has exactly three variables (negated or not negated) the formula is in a special
called 3-SAT. An example 3-SAT formula is,

$$
(a \lor b \lor \neg c) \land ( \neg a \lor e \lor \neg f) \land ( f \lor \neg a \lor c)
$$

This formula has satisfying assignment {a:True, f: False, c: True}.

 

In [1]:
### how do we represent this in python?

## for a variable named v, represent it either (v,0) or (v,1), if it is not negated or negated.
## a clause is a list of such variable
## a formula is a list of such clauses

A = [('a',0),('b',0),('c',1)]
B = [('a',1),('e',0),('f',1)]
C = [('f',0),('a',1),('c',0)]

formula = [A, B, C]
print(formula)
satisfying_assignment = {'a':True, 'f': False, 'c': True}

def evaluate_cnf(value, assignment):
    
    def eval_disjunct_helpe(disjunct):
        for variable in disjunct:
            if variable[0] in assignment:
                return True
            if variable[1] == 1 and not assignment[variable[0]]:
                return True
        return False
    
    for clause in formula:
        if not eval_disjunct_helpe(clause): return False
    return True

evaluate_cnf(formula,satisfying_assignment)
        

[[('a', 0), ('b', 0), ('c', 1)], [('a', 1), ('e', 0), ('f', 1)], [('f', 0), ('a', 1), ('c', 0)]]


True


### 3-SAT is in NP

We give an exponential algorithm for solving 3-SAT:

- Given a problem with $n$ clauses, collect the $m$ variables appearing in the clauses, $m<=3n$.
- Enumerate all possible $2^m$ ways of assignment True/False to the $m$ variables.
- For each possibility, check whether that assignment is a satisfying assignment.

Simple and obvious, this is called generally a solution by _brute force_. It runs in time $O((3n) 2^{m})$.
For each of the exponential number of assignments, run the above algorithm (which is linear - basically
sweep through an object $n$ in length making a local decision based on variable triples) 
to determine if the assignment is a satisfying assignment.

This algorithm is exponential in the problem size. That does not mean that an solution to 3-SAT
must be an exponential algorithm. That would be the _complexity_ of the problem.
The exact complexity of 3-SAT is unknown, except that it is no worse than exponential.

Note however, the structure of the solution. It contains a polynomial time algorithm that given
a satisfying assignment returns true, but if the assignment is not satisfying, the algorithm 
returns False. The problem may or may not be polynomial time solable, but it is polynomial time
_verifiable_.

The collection of problems that are polynomial time verifiable is called NP, non-deterministic
polynomial time.




### A slightly different algorithm

The 3-SAT solution adopted here improves on bruit search, but is still exponential time.

The search scans the clauses in the order presented, building up a collection of 
truth assignments to make each clause true as it is encountered.

The code uses recursion based on attempting to make a particular clause evaluate true, indicated
by the variable in the code "clause_number", and the recurse in clause_number+1. The result from the
recursion is either True, meaning the recursion managed to complete the variable assignment to make
clause clause_number+1 and all following clauses evaluate true, or returns False, meaning it could not.

If True, since the clause at clause_number has been made to evaluate true, true is returned.

If False, then a different way to make clause_number true is tried. If after trying all ways to make
clause_number true, and for none of them does the recursive call for clause_number+1 returns True, return
False.
 
   

In [2]:
###
### exact 3-SAT solver
### author: bjr
### date: 10 apr 2020
###

# last update: 10 apr 2020


# problem statement: 

# given a set of clauses, where is clause is three un-negated or negated variables
# find a truth assignment to the variables such that every clause has at least one 
# member being true - i.e. an un-negated variable set to True, or a negated variable
# set to False

# this is called a statisfying assignment. 


class Exact3SAT:

    # data structures

    # clause list:
    #   clause := [(v_1,0|1), (v_2,0|1), (v_3,0|1)]
    #   where v_i are string type variable names, and 0 for unnegated, 1 for negated
    #   clause_list := [ clause, clause, .., clause ]

    # value dictionary:
    # a dictionary of value assignents { .... variable | (True|False)  ...}

    def __init__(self,clause_list,verbose=0):
        self.values = {}
        self.clause_list = clause_list
        self.clause_list_len = len(clause_list)
        self.verbose = verbose

    def get_assignment(self):
        return self.values

    def is_satisfied(self,clause):
        return self.is_satisfied_aux(clause,self.values)
        
    def is_satisfied_aux(self,clause,values):
       
        for var in clause:
            if var[0] in values:
                if values[var[0]]==True and var[1]==0:
                    if self.verbose>0: print("is_satisfied: RETURNS True,\n\tinput", clause, values )
                    return True
                if values[var[0]]==False and var[1]==1:
                    if self.verbose>0: print("is_satisfied: RETURNS True,\n\tinput", clause, values )
                    return True
        if self.verbose>0: print("is_satisfied: RETURNS False,\n\tinput", clause, values )
        return False
    
    def solve(self):
        return self.search_aux(0)

    def search_aux(self,clause_number):
        if self.verbose>0: print('search_aux: ENTERED (clause number, values)\n\t', clause_number, self.values)
        
        if clause_number>=self.clause_list_len:
            return True
            
        clause = self.clause_list[clause_number] 
        if self.is_satisfied(clause):
            return self.search_aux(clause_number+1)

        for j in range(3):
            var = clause[j][0]
            if var not in self.values:
                self.values[var] = True if clause[j][1] == 0 else False
                if self.verbose>1: print("search_aux: TRYING a setting (clause, var, setting)\n\t",j,var,self.values[var])
                if self.search_aux(clause_number+1):
                    return True
                del self.values[var]

        # could not find a variable to set
        return False    
        
    def verify(self,cnf,values):
        for clause in cnf:
            if not self.is_satisfied_aux(clause,values):
                return False
        return True
            
        
    

### Test functions



In [3]:
from random import randint 

class Exact3SAT_Test:
    
    @staticmethod
    def run_3sat(cnf,verbose=0):
        three_sat = Exact3SAT(cnf,verbose)
        result = three_sat.solve()
        assignment = three_sat.get_assignment()
        print(cnf, "\n\t", result, assignment)
        # check
        assert( result == three_sat.verify(cnf,assignment))

    @staticmethod
    def random_cnf3_3var(n):
        l = []
        for i in range(n):
            l += [[('x',randint(0,1)),('y',randint(0,1)),('z',randint(0,1))]]
        return l

    @staticmethod
    def random_cnf3_4var(n):
        l = []
        var_name = [ 'x', 'y', 'z', 't' ]
        for i in range(n):
            l += [[
                    (var_name[randint(0,3)],randint(0,1)),
                    (var_name[randint(0,3)],randint(0,1)),
                    (var_name[randint(0,3)],randint(0,1))
                 ]]
        return l

    @staticmethod
    def bruit_force_3var(three_sat, cnf):
        # try to solve it by bruit force
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    values = { 'x':bool(x), 'y':bool(y), 'z':bool(z) }
                    if three_sat.verify(cnf,values):
                        return values
        return {}

    @staticmethod
    def bruit_force_4var(three_sat, cnf):
        # try to solve it by bruit force
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    for t in range(2):
                        values = { 'x':bool(x), 'y':bool(y), 'z':bool(z), 't':bool(t) }
                        if three_sat.verify(cnf,values):
                            return values
        return {}

    @staticmethod
    def fuzz_3sat_3var(n,m,verbose=0):

        verdict = True
        for i in range(n):
            random_cnf = Exact3SAT_Test.random_cnf3_3var(m)
            three_sat = Exact3SAT(random_cnf,verbose)
            result = three_sat.solve()
            assignment = three_sat.get_assignment()
            if verbose>0 : print(random_cnf, "\n\t", result, assignment)
            if result:
                v = three_sat.verify(random_cnf, assignment)
                verdict  = verdict and v
                if verbose>0 : print("\tverified:", v)
            else:
                v = Exact3SAT_Test.bruit_force_3var(three_sat, random_cnf)
                verdict = verdict and len(v)==0
                if verbose>0 : print("\tbruit force: ", v)
        return verdict

    @staticmethod
    def fuzz_3sat_4var(n,m,verbose=0):
        
        verdict = True
        for i in range(n):
            random_cnf = Exact3SAT_Test.random_cnf3_4var(m)
            three_sat = Exact3SAT(random_cnf,verbose)
            result = three_sat.solve()
            assignment = three_sat.get_assignment()
            if verbose>0 : print(random_cnf, "\n\t", result, assignment)
            if result:
                v = three_sat.verify(random_cnf, assignment)
                verdict  = verdict and v
                if verbose>0 : print("\tverified:", v)
            else:
                v = Exact3SAT_Test.bruit_force_4var(three_sat, random_cnf)
                verdict = verdict and len(v)==0
                if verbose>0 : print("\tbruit force: ", v)
        return verdict
            



### Run tests


1. A few hand crafted CNF's are tried.
2. Random 3-SAT's are generated over variables x, y and z. The 3-SAT solver solves the random 3-SAT, and
   the result is verified either by varifing a positive result, or bruit-search for a solution to confirm
   a negative result.
3. Random 3-SAT's are generated over the four variables x, y, z and t, and the test is the same as above.



In [4]:
            
cnf3_1 = [ [('x',0),('y',0),('z',0)] ]
cnf3_2 = [ [('x',1),('y',0),('z',0)] ]
cnf3_3 = [ [('x',0),('y',0),('z',0)], [('x',1),('y',0),('z',0)] ]
cnf3_4 = [ [('x',1),('y',0),('z',0)], [('x',0),('y',1),('z',0)] ]
cnf3_5 = [ [('x',0),('x',0),('x',0)], [('x',1),('x',1),('x',1)] ]
cnf3_6 = [ [('x',0),('x',0),('x',0)], [('y',1),('y',1),('y',1)], 
           [('x',1),('y',0),('z',0)], [('z',1),('z',1),('z',1)] ]
cnf3_7 = [ [('x',0),('x',0),('x',0)], [('y',1),('y',1),('y',1)], 
           [('x',1),('y',0),('z',0)], [('x',0),('z',1),('z',1)] ]
 
cnfs = [ cnf3_1, cnf3_2, cnf3_3, cnf3_4, cnf3_5, cnf3_6, cnf3_7]

for cnf in cnfs:
    Exact3SAT_Test.run_3sat(cnf)
    
print("Test: fuzz_3sat_3var\n\tverdict = ", Exact3SAT_Test.fuzz_3sat_3var(7,20))
print("Test: fuzz_3sat_4var\n\tverdict = ", Exact3SAT_Test.fuzz_3sat_4var(7,20))

[[('x', 0), ('y', 0), ('z', 0)]] 
	 True {'x': True}
[[('x', 1), ('y', 0), ('z', 0)]] 
	 True {'x': False}
[[('x', 0), ('y', 0), ('z', 0)], [('x', 1), ('y', 0), ('z', 0)]] 
	 True {'x': True, 'y': True}
[[('x', 1), ('y', 0), ('z', 0)], [('x', 0), ('y', 1), ('z', 0)]] 
	 True {'x': False, 'y': False}
[[('x', 0), ('x', 0), ('x', 0)], [('x', 1), ('x', 1), ('x', 1)]] 
	 False {}
[[('x', 0), ('x', 0), ('x', 0)], [('y', 1), ('y', 1), ('y', 1)], [('x', 1), ('y', 0), ('z', 0)], [('z', 1), ('z', 1), ('z', 1)]] 
	 False {}
[[('x', 0), ('x', 0), ('x', 0)], [('y', 1), ('y', 1), ('y', 1)], [('x', 1), ('y', 0), ('z', 0)], [('x', 0), ('z', 1), ('z', 1)]] 
	 True {'x': True, 'y': False, 'z': True}
Test: fuzz_3sat_3var
	verdict =  True
Test: fuzz_3sat_4var
	verdict =  True


### Reducing general SAT to 3-SAT.


Since a 3-SAT is a SAT, an algorithm to solve SAT (in general) will solve our 3-SAT's. 
However, is SAT even harder than 3-SAT? It is not. But how to you prove that it is not.
By showing that, despite appearances, SAT is also a special case of 3-SAT.

Here's the secret.

There is a mechanical transformation from any SAT to a 3-SAT that,

- Does not appreciably increase the length of the instance,
- Is a 3-SAT which is True when the original SAT is True,
- and False when the original is False.

This is called a _polynomial reduction from SAT to 3-SAT_ and has the symbol SAT $\le_{poly}$ 3-SAT.
It is formaly defined as a polynomail time, true preserving function,

$$
R:\mbox{ SAT }\longrightarrow\mbox{ 3-SAT }
$$

And it is useful because whenever there is a polynomial verifier for 3-SAT, $V_\mbox{3-SAT}$, 
there is a polynomial verifier for SAT, $V_\mbox{SAT}$ defined as,

$$
V_\mbox{SAT} = V_\mbox{3-SAT}\circ R
$$

which takes the original problem, applies $R$ in polynomial time, adds the application of 
$V_\mbox{3-SAT}$ in polynimial time, in total runs in polynomial time, and answers correctly
True or False.

